In this notebook we prepare a csv file to create the demi violin : 

In [146]:
# Third parties
import pandas as pd

# Native

# Custom

# Classes
from scipy.stats import gaussian_kde

# Functions
from numpy import histogram, linspace, savetxt, array

In [40]:
OG = pd.read_csv('../../data/2025-01-07-2024-10-29-quinquadef4-neat-abstract-bert.csv')

/var/folders/xs/d90v1vkn16db_7z493h6kwt40000gn/T/ipykernel_63037/2325017633.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  OG = pd.read_csv('../../data/2025-01-07-2024-10-29-quinquadef4-neat-abstract-bert.csv')


In [41]:
df = OG.loc[:,['annee','bert_genre','revue']]
df = df.sort_values('annee')
df.index = range(len(df))
df

,annee,bert_genre,revue
0,2001,False,Revue d’histoire moderne & contemporaine
1,2001,False,Revue française de science politique
2,2001,False,Revue française de science politique
3,2001,False,Revue française de science politique
4,2001,True,Annales historiques de la Révolution française
...,...,...,...
47156,2023,False,Mondes en développement
47157,2023,False,Participations
47158,2023,False,Participations
47159,2023,False,Participations


In [42]:
df.loc[(df['annee'] == 2001) & (df['revue'] == 'Clio'),:]

,annee,bert_genre,revue
917,2001,True,Clio
918,2001,True,Clio
919,2001,True,Clio
946,2001,True,Clio
948,2001,False,Clio
959,2001,True,Clio
960,2001,True,Clio
961,2001,True,Clio
962,2001,True,Clio
963,2001,True,Clio


In [43]:
# Create a dataframe
'''
annee | revue | proportion
'''

years = set(df['annee'])

year_vector = []
revue_vector = []
proportion_vector = []

for year in years : 
    set_of_revues = set(
        df.loc[
            df['annee'] == year,
            'revue'
            ]
    )
    for revue in set_of_revues:
        year_vector.append(year)
        revue_vector.append(revue)
        proportion_vector.append(
            df.loc[
                (df['annee'] == year) & (df['revue'] == revue),
                'bert_genre'
            ].dropna().mean()
        )

    


In [47]:
df_to_histogram = pd.DataFrame({
    'year' : year_vector,
    'revue' : revue_vector,
    'proportion' : proportion_vector
})
df_to_histogram.to_csv('../../data/checkpoints/demi_violin_setup.csv')

In [48]:
df_to_histogram

,year,revue,proportion
0,2001,Clio,0.952381
1,2001,Ethnologie française,0.057692
2,2001,Le Mouvement Social,0.000000
3,2001,Cahiers internationaux de sociologie,0.111111
4,2001,Études rurales,0.000000
...,...,...,...
1972,2023,Sciences sociales et santé,0.333333
1973,2023,"Parlement[s], Revue d'histoire politique",0.083333
1974,2023,Quaderni,0.000000
1975,2023,Cahiers d'études africaines,0.086957


In [53]:
histogram(df_to_histogram.loc[
    df_to_histogram['year'] == 2001,'proportion'
], bins = linspace(0,1,10))

(array([33,  5,  1,  0,  0,  0,  0,  0,  3]),
 array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]))

In [82]:
class custom_histogram :
    def __init__(self, df : pd.DataFrame, nbins : int = 10):
        self.dataframe = df
        self.year_vector = set(df['year'])
        self.nbins = nbins
        self.density = True
        self.FOLDERNAME = '../../data/checkpoints/'
        
    def run_and_save(self, filename : str = 'histogram_for_d3js.csv'):
        '''
        create a matrix : 
        ---------------------> histogram bins
        |
        |
        |
        |
        v year
        
        the value M[year, histogram_bin] = proportion
        to make it easier to use with d3js we are saving it : 
        (x,y,z) -> (histogram bin, density, year)
        '''
        bins_array = []
        density_array = []
        year_array = []

        for year in self.year_vector:
            density, bins = histogram(
                self.dataframe.loc[
                    self.dataframe['year'] == year,'proportion'
                ], bins = linspace(0,1, self.nbins)
            )

            if self.density:
                density = density / sum(density)
            
            for d, h_bin in zip(density,bins):
                year_array.append(year)
                density_array.append(d)
                bins_array.append(h_bin)

        pd.DataFrame({
            'bins' : bins_array ,
            'density' : density_array,
            'year' : year_array,
        }).to_csv(self.FOLDERNAME + filename)


In [163]:
CH = custom_histogram(df_to_histogram, 10)
CH.density = True
CH.run_and_save()

In [137]:
df_from_histogram = pd.read_csv(CH.FOLDERNAME + filename)

In [138]:
kernel = gaussian_kde(
    df_from_histogram.loc[
        df_from_histogram['year'] == 2001, 'density'
        ].to_numpy(),
        bw_method = 'scott')

In [161]:
class KERNEL:
    def __init__(self, filename : str = 'histogram_for_d3js.csv'):
        self.filename = filename
        self.FOLDERNAME = '../../data/checkpoints/'
        self.method = 'scott'
        self.n = 100

    def run_and_save(self, filename : str = 'kernel_for_d3js.csv'):
        '''
        create a matrix : 
        ---------------------> x
        |
        |
        |
        |
        v year
        
        the value M[year, histogram_bin] = proportion
        to make it easier to use with d3js we are saving it : 
        (x,y,z) -> (x, density, year)
        '''
        # create an x vector 
        X = linspace(0,1,self.n)

        # read dataframe
        dataframe = pd.read_csv(self.FOLDERNAME + self.filename)

        # get all the years : 
        years_set = set(dataframe['year'])
        
        density_vector = []
        
        # run
        for year in years_set:
            print(year)

            kernel = gaussian_kde(
                dataframe.loc[
                   dataframe['year'] == year, 'density'
                ].to_numpy(), bw_method = self.method
            )
            density_vector.append(
                list(kernel(X))
                )
        savetxt(
            self.FOLDERNAME + filename,
            array(
                density_vector,
                dtype = float
            ),
            delimiter = ','
        )

In [165]:
k = KERNEL()
k.run_and_save()

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
